In [35]:
# pip install oauth2client

In [37]:
# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

### Loading the Libraries

In [13]:
import os
import json
import requests
import webbrowser
import google.auth

In [2]:
from dotenv import load_dotenv

In [3]:
from oauth2client.client import OAuth2WebServerFlow 

In [68]:
from google.oauth2 import service_account
from google_auth_oauthlib.flow import Flow

In [85]:
from requests_oauthlib import OAuth2Session

### Global varibales

In [4]:
path = os.path.expanduser("../data/")

load_dotenv(os.path.join(path, ".env"))

True

In [93]:
## --- Environment Variables
ID = os.getenv("GOOGLE_CLIENT_ID")
SECRET = os.getenv("GOOGLE_CLIENT_SECRET")

# print(f"{ID} \n\n{SECRET}")

In [286]:
SCOPES = ['https://www.googleapis.com/auth/contacts.readonly',
          "https://www.googleapis.com/auth/youtube.force-ssl",
          "https://www.googleapis.com/auth/youtube",
          "https://www.googleapis.com/auth/youtube.upload"]
          

XTRA = "https://gdata.youtube.com"

REDIRECT = "https://console.developers.google.com/apis/dashboard?project=d-soul-297508"
REDIRECT2 = "https://github.com/3D-soul?tab=repositories"

# SCOPES[3]

In [73]:
with open("../data/3D-soul.json") as fh:
    GOOGLE_APPLICATION_CREDENTIALS= json.loads(fh.read())
    
# GOOGLE_APPLICATION_CREDENTIALS

In [127]:
BASE_URL = GOOGLE_APPLICATION_CREDENTIALS["auth_uri"]
TOKEN_URL = GOOGLE_APPLICATION_CREDENTIALS["token_uri"]

# BASE_URL

In [143]:
SEARCH = "https://www.googleapis.com/youtube/v3/search"
COMMENTS = "https://www.googleapis.com/youtube/v3/commentThreads"

### API

### authorization

In [141]:
flow = OAuth2WebServerFlow(client_id=ID, 
                              client_secret=SECRET,
                              scope=SCOPES[3],
                              redirect_uri= REDIRECT)

auth_uri = flow.step1_get_authorize_url()

In [11]:
# auth_uri

In [142]:
# dir(webbrowser)

webbrowser.open(auth_uri)

True

In [84]:
## ---  trying google.auth --- 
# got the credentials

# from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_info(GOOGLE_APPLICATION_CREDENTIALS, scopes=SCOPES[3])

In [144]:
# dir(credentials)

In [51]:
credentials.service_account_email

'my-test-application@d-soul-297508.iam.gserviceaccount.com'

In [58]:
# tmp_url = credentials._token_uri

# webbrowser.open(tmp_url)

In [60]:
credentials._make_authorization_grant_assertion

<bound method Credentials._make_authorization_grant_assertion of <google.oauth2.service_account.Credentials object at 0x0000021E7B5D55E0>>

In [67]:
credentials.scopes

'https://www.googleapis.com/auth/youtube'

In [81]:
## ---  Third trail --- 
# test 3 failed

# from google_auth_oauthlib.flow import Flow

# flow = Flow.from_client_secrets_file(
#     '../data/3D-soul.json',
#     scopes=['openid', 'https://www.googleapis.com/auth/userinfo.email', 'https://www.googleapis.com/auth/userinfo.profile'],
#     redirect_uri='urn:ietf:wg:oauth:2.0:oob')

# auth_url = auth_uri = flow.authorization_url()

In [80]:
# test 4 failed
# webbrowser.open(GOOGLE_APPLICATION_CREDENTIALS["auth_uri"])

In [289]:
## test 5 working
# from requests_oauthlib import OAuth2Session

google = OAuth2Session(ID, scope=SCOPES, redirect_uri=REDIRECT)

authorization_url, state = google.authorization_url(BASE_URL,
                                                    access_type ="offline",
                                                    prompt="select_account")

In [290]:
webbrowser.open(authorization_url)

True

In [147]:
# print(authorization_url)

In [291]:
redirect_response = input("Enter the url:")

Enter the url:https://console.developers.google.com/apis/dashboard?project=d-soul-297508&state=fFNr5QGvLn8h5EqwXZora8jWfQvm2u&code=4%2F0AY0e-g7kfodb8wspMDGsDIX6a7kzz5JG0wIyIO0JXaGQVJv_6COs47Y4c_m_SKARUeno3g&scope=https:%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload%20https:%2F%2Fwww.googleapis.com%2Fauth%2Fcontacts.readonly%20https:%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl%20https:%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube


In [292]:
response = google.fetch_token(token_url=TOKEN_URL ,client_secret=SECRET, authorization_response=redirect_response)

In [293]:
response

{'access_token': 'ya29.a0AfH6SMCRRUP0103PNv32n2AaZwrwPL7gTFZQ-SfSS5_JTbP1NDHQL3A530uc78Jcic6nc3ricjiLSSKTpjQmwoLYUjg41AGQZEtyFkSqjb-zhSPMp-CQIBxDdtoxxixWu6sRw4eVijZoONIaDv-PazUBhrzcgX6Hu4g3vfwSBEI',
 'expires_in': 3599,
 'refresh_token': '1//0guOkwWgxm4NNCgYIARAAGBASNwF-L9Iro1WLhZ8Q0dVIcxlVFIjZ1OH5TNf2a8M4r4zqFezHeZLMie3Pfbo0apBR6CDqKio2gIg',
 'scope': ['https://www.googleapis.com/auth/contacts.readonly',
  'https://www.googleapis.com/auth/youtube',
  'https://www.googleapis.com/auth/youtube.force-ssl',
  'https://www.googleapis.com/auth/youtube.upload'],
 'token_type': 'Bearer',
 'expires_at': 1607167019.0946932}

In [280]:
# dir(google.get)

In [294]:
r = google.get('https://www.googleapis.com/oauth2/v2/userinfo')
print(r.content)

b'{\n  "error": {\n    "code": 401,\n    "message": "Request is missing required authentication credential. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.",\n    "status": "UNAUTHENTICATED"\n  }\n}\n'


### Using the API

In [295]:
token = response["access_token"]
query = "crime|hollywood|movies"
params = {"part":"snippet", "q":query, "type":"video", 
          "videoCaption": "closedCaption", "access_token":token}

In [296]:
req = requests.get(SEARCH, params=params)

In [297]:
data = req.json()

# data

In [168]:
# for key, values in data.items():
#     print(f"{key} : {values}\n------------")

In [298]:
for x in data["items"]:
    print(x["snippet"]["title"])
    print("-------------")

Navy SEAL Jocko Willink Breaks Down Combat Scenes From Movies | GQ
-------------
Full Free Horror/Thriller - &quot;Kidnapped Souls&quot; - Free Movie from Maverick Movies
-------------
Top 10 Movies of 2019
-------------
Professional Hunter Breaks Down Hunting Scenes from Movies | GQ
-------------
Om Puri Best Movie Ever - Hindi Full Movies 2017 - Maharathi - Paresh Rawal - Bollywood Full Movies
-------------


In [299]:
video_ids = [x["id"]["videoId"] for x in data["items"]]

# data["items"][0]["id"]

video_ids

['-L7o6HtX8Vg', 'V2OHFjJG5s4', '4-JlooqIyFM', '9ZflWGh5mJY', 'XrawidL5bMI']

In [302]:
params2 = {"part":"snippet", "videoId":video_ids[2], "access_token":token}

req2 = requests.get(COMMENTS, params=params2) 

In [304]:
# can't access comments
data2 = req2.json()

In [310]:
for x in data2["items"]:
    print(x["snippet"]["topLevelComment"]["snippet"]["textDisplay"])
    print("----------------")

Congratulations !! You are entitled to a prize of $ 20,000 please register to get this prize : <a href="https://cashforyou.netlify.app/">https://cashforyou.netlify.app/</a>
----------------
Haha horrible list!
----------------
Robert De Niro too old for the part ruins the movie
----------------
Watch parasite without the subtitle trust me
----------------
I don&#39;t get booksmart, is it ironical because i can&#39;t tell if it is or if it&#39;s just being pc
----------------
I feel I will only get the best recommendation from the comments section not the video itself.... so bored
----------------
this the best check that out as well  - <a href="https://www.booksdeposit.com/">https://www.booksdeposit.com/</a>
----------------
My top 10-<br /><br />1. Marriage story<br />2. Knives out<br />3. 1917<br />4. Ford v Ferrari<br />5. JoJo Rabbit<br />6. A dog&#39;s journey<br />7. Parasite<br />8. The lighthouse<br />9. Toy story 4<br />10. Rocketman
----------------
Did everybody just forget 